## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:$G$的最小生成树（MST）必须包含$e_1$和$e_2$，但不一定包含$e_3$。

（反证法）

假设$G$有最小生成树（MST）$T$，且$T$的边集未包含$e_1$。先在$T$上添加边$e_1$，根据生成树的定义必会产生一个包含边$e_1$的环，接着在此环上除掉除边$e_1$以外的边$e_i$，产生一个新的生成树$T1$，而$weight(e_1) < weight(e_i)$，那么$Σweight(T1) < Σweight(T)$，此时$T$并非最小生成树，与假设条件矛盾，所以$G$的最小生成树（MST）$T$必须包含$e_1$。

在证明必须包含边$e_1$的反证过程中，将生成包含边$e_1$的环当作$G$的一个子图$G'$，

边$e_2$若不处于$G'$，则边$e_2$是$T$除去$G'$以外权重最小的边，此时同理能够证明$T$必须包含边$e_2$，同样地，边$e_3$若不处于证明必须包含$e_2$和$e_1$时生成的环，那么此时$T$必须包含边$e_3$；

边$e_2$若处于子图$G'$，要使新产生的生成树$T1$的权重和最小，只有除去$G'$中权重最大的边$e_{g'max}$，而子图$G'$至少由三条边构成，那么$e_2$一定不为$e_{g'max}$，因此$T$必须包含边$e_2$，若此时子图$G'$还包含边$e_3$，那么在$G'$只包含三条边的情况下，$e_3$为$e_{g'max}$，那么除去的是$e_3$，$T$则不包含边$e_3$。

综上，$G$的最小生成树（MST）必须包含$e_1$和$e_2$，但不一定包含$e_3$。


In [ ]:
# 借助大模型产生最小生成树
def prim_mst(matrix):
    V = len(matrix)  # 顶点数
    if V == 0:
        return 0, []
    
    # 初始化数据结构
    key = [float('inf')] * V   # 记录顶点到MST的最小边权
    parent = [-1] * V          # 记录MST中的父节点
    selected = [False] * V     # 标记顶点是否已加入MST
    
    # 选择起始点（这里默认选0，也可以改为其他顶点）
    key[0] = 0
    
    for _ in range(V):
        # 线性搜索：找到未选顶点中key最小的顶点u
        u = -1
        min_key = float('inf')
        for v in range(V):
            if not selected[v] and key[v] < min_key:
                min_key = key[v]
                u = v
        
        if u == -1:  # 图不连通时可能触发
            break
        
        selected[u] = True
        
        # 更新u的邻接顶点的key和parent
        for v in range(V):
            if matrix[u][v] > 0 and not selected[v]:
                if matrix[u][v] < key[v]:
                    key[v] = matrix[u][v]
                    parent[v] = u
    
    # 构建MST的边集合
    mst_edges = []
    total_weight = 0
    for v in range(1, V):  # 起点0没有父节点，跳过
        if parent[v] != -1:
            mst_edges.append((parent[v], v, matrix[parent[v]][v]))
            total_weight += matrix[parent[v]][v]
    
    return total_weight, mst_edges


# 最小生成树必须包含$e_3$的反例
G = [[0, 1, 3, 0],
     [1, 0, 2, 0],
     [3, 2, 0, 4],
     [0, 0, 4, 0]]

total_weight, edges = prim_mst(G)
print(f"最小生成树总权重: {total_weight}")
print("边集合:")
for u, v, w in edges:
    print(f"{u} - {v} (权重 {w})")

# W(T) = {1, 2, 4} 此时不包含权重第三小的边。

最小生成树总权重: 7
边集合:
0 - 1 (权重 1)
1 - 2 (权重 2)
2 - 3 (权重 4)


## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

answer1（反证法）

假设$G$ 的某个最小生成树（MST）$T1$不是瓶颈生成树（BST）$T$，

根据瓶颈生成树（BST）的定义，则有$T1$最大权重边$w(e1_{max}) > T$最大权重边$w(e_{max})$，且$T$中的每条边权重都小于$w(e1_{max})$，则用瓶颈生成树$T$中使$T1$的$e1_{max}$两端的节点互通的边集合任选一条边替换$T1$的$e1_{max}$，此时新产生的生成树的权重和会比$T1$的更小，以此与假设条件矛盾，故$G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）。




idea for 2：
将图$G$中所有超过$b$的边删除生成图$G'$，然后判断图$G'$是否仍为连通图，若是则图$G$一定有不超过$b$的瓶颈生成树，反之则无。（借用大模型构建邻接表表示带权重的无向图$G(V,E,w)$）

In [4]:
# add your code here
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[] for _ in range(vertices)]
    
    def add_edge(self, u, v, w):
        # Since it's an undirected graph, add edges for both directions
        self.graph[u].append((v, w))
        self.graph[v].append((u, w))
    
    def print_graph(self):
        for i in range(self.V):
            print(f"Vertex {i}:", end=" ")
            for neighbor, weight in self.graph[i]:
                print(f"-> ({neighbor}, weight={weight})", end=" ")
            print()

def remove_edges_above_weight(g, b):
    """移除图中权重大于b的边，返回新图"""
    new_graph = Graph(g.V)  # 初始化邻接表
    
    for u in range(g.V):
        for v, w in g.graph[u] : 
            if w <= b:
                new_graph.add_edge(u, v, w)
    
    return new_graph

def is_connected(g):
    """用DFS判断图g是否连通"""
    if g.V == 0:
        return True
        
    visited = [False] * g.V
    
    def dfs(v):
        visited[v] = True
        for neighbor, _ in g.graph[v]:
            if not visited[neighbor]:
                dfs(neighbor)
    
    # 从顶点0开始遍历
    dfs(0)
    
    # 保证遍历到了所有顶点
    return all(visited)
   
# 示例
g = Graph(4)
g.add_edge(0, 1, 1)  # Edge 0-1 with weight 1
g.add_edge(1, 2, 2)  # Edge 1-2 with weight 2
g.add_edge(2, 3, 4)  # Edge 2-3 with weight 4
g.add_edge(0, 2, 3)  # Edge 0-2 with weight 3

print("Graph structure:")
g.print_graph()

# 测试
test_b_values = [2, 3, 4, 5]
for b in test_b_values:
    result = is_connected(remove_edges_above_weight(g,b))
    print(f"\nBottleneck spanning tree with max weight <= {b}: {result}")


# algorithm of the liear time complexity : O(E+V)

Graph structure:
Vertex 0: -> (1, weight=1) -> (2, weight=3) 
Vertex 1: -> (0, weight=1) -> (2, weight=2) 
Vertex 2: -> (1, weight=2) -> (3, weight=4) -> (0, weight=3) 
Vertex 3: -> (2, weight=4) 

Bottleneck spanning tree with max weight <= 2: False

Bottleneck spanning tree with max weight <= 3: False

Bottleneck spanning tree with max weight <= 4: True

Bottleneck spanning tree with max weight <= 5: True


idea for 3：按照问题2删除边的思路，对图$G$的边权重从大到小依次删除边，若删除的边产生了非连通图，则保留该边，继续删除下一条边，直至边的数量刚好是顶点数-1，那么生成的子图$G''$一定是瓶颈生成树。

为了达到线性时间复杂度，将sort()函数换用计数排序算法将边权重从大到小的排序时间复杂度降成O(E)，并且用并查集的fine和union操作来判断图的连通情况，将删除边后对图的连通性判断的时间复杂度近似于O(E)。


In [5]:
# add your code here
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[] for _ in range(vertices)]
    
    def add_edge(self, u, v, w):
        # Since it's an undirected graph, add edges for both directions
        self.graph[u].append((v, w))
        self.graph[v].append((u, w))
    
    def print_graph(self):
        for i in range(self.V):
            print(f"Vertex {i}:", end=" ")
            for neighbor, weight in self.graph[i]:
                print(f"-> ({neighbor}, weight={weight})", end=" ")
            print()

def linear_sort_edges(edges):
    """用计数排序达到线性时间复杂度"""
    # 获取最大权重
    max_weight = max(w for _, _, w in edges)
    
    # 初始化计数数组
    count = [0] * (max_weight + 1)
    
    # 计算每个元素的出现次数
    for _, _, w in edges:
        count[w] += 1
        
    # 计算每个元素的起始索引
    for i in range(1, len(count)):
        count[i] += count[i - 1]
        
    # 初始化输出数组
    output = [None] * len(edges)
    # 构建输出数组
    for i in range(len(edges) - 1, -1, -1):
        u, v, w = edges[i]
        output[count[w] - 1] = (u, v, w)
        count[w] -= 1

    # 逆排序输出数组
    return output[::-1]
        
        
def find_bst(g):
    # 存储所有边和权重
    edges = []
    for u in range(g.V): 
        for v, w in g.graph[u]:
            if u < v:  # 避免重复边
                edges.append((u, v, w))
    
    # 按权重从大到小排序边
    # edges.sort(key=lambda x: -x[2]) # O(ElogE)
    edges = linear_sort_edges(edges) #  O(E）
    
    # 用并查集跟踪连通分量
    parent = list(range(g.V)) #O(V)
    
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]
    
    def union(x, y):
        parent[find(x)] = find(y)
    
    # 存储BST的边
    bst_edges = []
    max_edge_weight = 0
    
    # 从最大权重边开始，删除不会导致图不连通的边
    for u, v, w in edges:
        # 如果删除这条边会导致图不连通，则保留它
        if find(u) != find(v):
            union(u, v)
            bst_edges.append((u, v, w))
            max_edge_weight = max(max_edge_weight, w)
            
        if len(bst_edges) == g.V - 1:
            break
            
    return max_edge_weight, bst_edges

# Test example
g = Graph(4)
g.add_edge(0, 1, 1)  # Edge 0-1 with weight 1
g.add_edge(1, 2, 2)  # Edge 1-2 with weight 2
g.add_edge(2, 3, 4)  # Edge 2-3 with weight 4
g.add_edge(0, 2, 3)  # Edge 0-2 with weight 3
print("Graph structure:")
g.print_graph()
# Test cases
print("\nBottleneck spanning tree:")
max_edge, edges = find_bst(g)
print(f"最大边权: {max_edge}")
print(f"边集合:{edges}")

# algorithm of the liear time complexity :O(E + V)

Graph structure:
Vertex 0: -> (1, weight=1) -> (2, weight=3) 
Vertex 1: -> (0, weight=1) -> (2, weight=2) 
Vertex 2: -> (1, weight=2) -> (3, weight=4) -> (0, weight=3) 
Vertex 3: -> (2, weight=4) 

Bottleneck spanning tree:
最大边权: 4
边集合:[(2, 3, 4), (0, 2, 3), (1, 2, 2)]


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


idea：对于此问题，需要计算添加每对候补的新道路$(u,v) \in E'$后s到t的最短距离，先是用Dijkstra算法计算s到u的最短距离d[s][u]，再构建反向图，用反向Dijkstra算法计算v到t的最短距离d[v][t]，最后对于每对候补路径(u,v)，得到s到t的最短距离是min(原始最短路径, d[s][u] + l'(u,v) + d[v][t])，从最短距离min中选择最小值对应的一对城市建立新道路。

正确性：要让新道路(u,v)的建立使s到t的距离缩短到最大，即是找到一对城市(u,v)，使得$s->u->v->t$的距离变得最小，用两次Dijkstra得到$s->u$和$v->t$的最短距离，再从$E'$得到$u->v$的距离$l'(u,v)$，组合$s->u->v->t$的距离与原始路径距离比较取较小值。

时间复杂度：此算法需要用到两次Dijkstra，而Dijkstra的时间复杂度是O(ElogV)，再加上还需要遍历候补的新路径(u,v)对应的最短距离，而$(u, v) \in E'$，则遍历的时间复杂度是O(E')，因此此算法的时间复杂度是O(ElogV+E')。

## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：

answer1:

证明：
将网络$G$中的每个顶点拆分成两个顶点入顶点vin和出顶点vout，再用一条容量为该顶点容量的边连接vin和vout（vin指向vout），接着将指向该顶点的边换成指向vin，该顶点指出的边换成从vout指出。

由此具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，其中$V' = 2*V = O(V) , E' = E + V = O(E) $

answer2:

算法设计：逃离问题要求路径之间不相交，也就是要限制路径上的顶点和边的容量都为1。

因此可以先将逃离网格构建成一个具有可比大小的普通流网络，设置一个源点s和一个汇点t，源点s连接所有的起点，所有的边界点连接汇点t，连接边的容量都设置为1，对每个网格点拆点分为vin和vout两个顶点，并在vin和vout中间连接一条容量为1的边，相邻格点间添加容量为1的边，以此生成流网络$G'$;

接着在流网络$G'$上用Ford-Fulkerson最大流算法，求出总的最大流值$f_max$，若$f_max = m(起点个数)$，则说明该逃离网格存在$ m $ 条顶点不相交的路径，反之则不存在。

时间复杂度：在此算法中，构建流网络$G'$的时间复杂度是$O(n^2)$，所用的Ford-Fulkerson最大流算法的时间复杂度是$O(mf_max)$，总的时间复杂度最大会是$O(n^2+m^2)$。
